In [58]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

In [59]:
data = pd.read_csv("aac_intakes_outcomes_address_income.csv", encoding="utf-8", low_memory=False)

In [60]:
data.head()

,age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,...,intake_weekday,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,age_upon_intake_number,age_upon_intake_lentime,found_location_street,zipcode,income
0,10 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,...,Thursday,14,1.0,0 days 14:07:00.000000000,0.588194,10,years,Colony Creek And Hunters Trace,78758,"47,244"
1,7 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,...,Friday,10,2.0,1 days 06:14:00.000000000,1.259722,7,years,8700 Research Blvd,78757,"62,992"
2,6 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2190,6.0,"(5.0, 7.5]",2014-03-08 17:10:00,...,Friday,14,3.0,1 days 02:44:00.000000000,1.113889,6,years,8700 Research,78757,"62,992"
3,10 years,A047759,2004-04-02 00:00:00,Partner,Transfer,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,...,Wednesday,15,1.0,4 days 23:17:00.000000000,4.970139,10,years,NaN,NaN,NaN
4,16 years,A134067,1997-10-16 00:00:00,NaN,Return to Owner,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,...,Saturday,9,1.0,0 days 02:52:00.000000000,0.119444,16,years,12034 Research Blvd,78759,"74,379"


In [61]:
#https://factfinder.census.gov/rest/communityFactsNav/nav?N=0&_t=1535644344663&log=t&searchTerm=78758
population_url = "https://factfinder.census.gov/rest/communityFactsNav/nav?spotlightId=POPULATION&searchTerm="
def getPOPULATIONbyZipCode(zipcode):
    try:
        response = requests.get(population_url + zipcode)
        if response.status_code == 200:
            result = response.json()
            return result["CFMetaData"]["measuresAndLinks"]["measure"]["value"]
        else:
            return None
    except:
        return None

In [62]:
zipcode_population = pd.DataFrame(data["zipcode"].unique(), columns=["zipcode"])
zipcode_population["population"] = None
zipcode_population.head()

,zipcode,population
0,78758,None
1,78757,None
2,NaN,None
3,78759,None
4,78751,None


In [63]:
getPOPULATIONbyZipCode("78733")

'8,762'

In [64]:
for index, row in zipcode_population.iterrows():
    zipcode = row["zipcode"]
    if zipcode != None:
        population = getPOPULATIONbyZipCode(zipcode)
        if population != "Data are not available for this topic and the selected geography":
            zipcode_population.loc[index, "population"] = population
            print(index, zipcode, population)
        else:
            zipcode_population.loc[index, "population"] = None
            print(index, zipcode, None)
    else:
        zipcode_population.loc[index, "population"] = None
        print(index, zipcode, None)
print("Done")

0 78758 44,072
1 78757 21,310
2 nan None
3 78759 38,891
4 78751 14,385
5 78739 16,792
6 78749 34,449
7 78744 42,820
8 78702 21,334
9 78735 16,131
10 78728 20,299
11 78617 22,210
12 78723 28,330
13 78727 26,689
14 78753 49,301
15 74051 1,904
16 78752 18,064
17 V6X 308,745,538
18 78722 5,901
19 78704 42,117
20 78750 26,814
21 78733 8,762
22 78621 21,659
23 78747 14,808
24 78748 40,651
25 78745 55,614
26 78705 31,340
27 78730 7,955
28 78731 24,614
29 78741 44,935
30 78660 68,789
31 78652 4,466
32 78746 26,928
33 78641 44,295
34 78721 11,425
35 78756 7,194
36 43065 38,821
37 78701 6,841
38 78724 21,696
39 78703 19,690
40 78729 27,108
41 78738 12,134
42 74058 4,259
43 78653 16,375
44 78736 6,946
45 78754 15,036
46 78712 860
47 78734 17,655
48 78645 9,773
49 78717 22,538
50 72762 35,342
51 78737 12,081
52 08003 30,629
53 10036 24,711
54 65616 24,284
55 78725 6,083
56 78719 1,764
57 78610 23,502
58 78742 820
59 74146 15,498
60 34747 13,692
61 74112 20,684
62 56370 308,745,538
63 75062 44,537


In [65]:
zipcode_population.to_csv("zipcode_population.csv", index=False)

In [67]:
zipcode_income = pd.read_csv("zipcode_population.csv")
zipcode_income.head()

,zipcode,population
0,78758,"44,072"
1,78757,"21,310"
2,78759,"38,891"
3,78751,"14,385"
4,78739,"16,792"


In [68]:
data_address_population = pd.merge(data, zipcode_population, how="left", left_on="zipcode", right_on="zipcode")
data_address_population.head()

,age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,...,intake_hour,intake_number,time_in_shelter,time_in_shelter_days,age_upon_intake_number,age_upon_intake_lentime,found_location_street,zipcode,income,population
0,10 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,...,14,1.0,0 days 14:07:00.000000000,0.588194,10,years,Colony Creek And Hunters Trace,78758,"47,244","44,072"
1,7 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,...,10,2.0,1 days 06:14:00.000000000,1.259722,7,years,8700 Research Blvd,78757,"62,992","21,310"
2,6 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2190,6.0,"(5.0, 7.5]",2014-03-08 17:10:00,...,14,3.0,1 days 02:44:00.000000000,1.113889,6,years,8700 Research,78757,"62,992","21,310"
3,10 years,A047759,2004-04-02 00:00:00,Partner,Transfer,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,...,15,1.0,4 days 23:17:00.000000000,4.970139,10,years,NaN,NaN,NaN,None
4,16 years,A134067,1997-10-16 00:00:00,NaN,Return to Owner,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,...,9,1.0,0 days 02:52:00.000000000,0.119444,16,years,12034 Research Blvd,78759,"74,379","38,891"


In [69]:
data_address_population.to_csv("aac_intakes_outcomes_address_income.csv", index=False)